Choose a collection of assets. Using simulation, find the weights that will lead to the minimum variance portfolio. Make a python function that will purchase the correct amount of assets on Alpaca.

In [32]:
pip install alpaca-trade-api

Note: you may need to restart the kernel to use updated packages.


In [33]:
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
import requests
import json
import alpaca_trade_api as tradeapi

# Choose a collection of assets

In [34]:
tickers = ['SLB',  'OXY', 'BP',  'WMB',  'BKR', 'APA', 'HES']

# Step 1 Get Optimal Weights


In [35]:
# 获取股票数据
data = yf.download(tickers, start='2000-01-01', end='2023-09-07')['Adj Close']

# 计算每只股票的日收益率
returns = data.pct_change().dropna()

# 计算协方差矩阵
cov_matrix = returns.cov()

# 定义目标函数，最小化方差
def objective(weights):
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    return portfolio_variance

# 设置约束条件，这里假设权重之和为1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# 设置初始权重猜测值
#一开始我是用的initial_weights = np.ones(len(tickers)) / len(tickers)
initial_weights = np.random.rand(len(tickers))
initial_weights /= initial_weights.sum() 

# 最小化目标函数，使用不同的优化方法
result = minimize(objective, initial_weights, constraints=constraints, method='trust-constr')


# 最优权重
optimal_weights = result.x

print("Optimal weights：", optimal_weights)
print("minimum variance：", result.fun)


[*********************100%%**********************]  7 of 7 completed


/Users/yuanhang/anaconda3/lib/python3.11/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Optimal weights： [0.26691407 0.18378235 0.04387213 0.20887486 0.09678252 0.01347171
 0.18630237]
minimum variance： 0.0005337423582006344


# Step 2 Get Account Balance

In [42]:
#与羊驼关联
API_KEY = "PKOK1AECOYSVDXHMQ7ON"
SECRET_KEY = "xeUuUHj4TuLJ95NaSSMbdKRWERyPGJofYbdoioki"
BASE_URL = "https://paper-api.alpaca.markets"
ACCOUNT_URL = BASE_URL + "/v2/account"
ORDERS_URL =  BASE_URL + "/v2/orders"

In [43]:
r = requests.get(ACCOUNT_URL, headers = {'APCA-API-KEY-ID': API_KEY,
                                         'APCA-API-SECRET-KEY':  SECRET_KEY})
info = json.loads(r.content)
accountval = float(info["cash"])

# Step 3 Get number of Shares

In [44]:
# 计算每个股票的股票数量
shares = []
for i, symbol in enumerate(tickers):
    weight = optimal_weights[i]
    price = data[symbol][-1]  # 获取最后一个交易日的股价
    qty = (weight * accountval) / price
    qty = int(qty)  # 取整数部分
    shares.append(qty)

for i, symbol in enumerate(tickers):
    print(f"The optimal number of stock {symbol} : {shares[i]}")

The optimal number of stock SLB : 4375
The optimal number of stock OXY : 2810
The optimal number of stock BP : 1129
The optimal number of stock WMB : 6254
The optimal number of stock BKR : 2600
The optimal number of stock APA : 301
The optimal number of stock HES : 1180


# Step 4 Purchase amounts

In [45]:
def create_order(symbol, qty, side, type_, time_in_force):
    data = {
            "symbol": symbol,
            "qty": qty,
            "side": side,
            "type": type_,
            "time_in_force": time_in_force
            }
    r = requests.post(ORDERS_URL, json=data, headers = {'APCA-API-KEY-ID': API_KEY, 
                                                        'APCA-API-SECRET-KEY':  SECRET_KEY})

    return json.loads(r.content)

for i, symbol in enumerate(tickers):
    qty = shares[i]

    create_order(symbol, qty, "buy", "market", "gtc")

# REBALANCE PORTFOLIO TO CHECK THAT THE WEIGHTS ARE ALIGNED EACH DAY

In [46]:
# 计算当前股票数量
current_shares = []
for i, symbol in enumerate(tickers):
    latest_prices = data.iloc[-1]
    price = latest_prices[symbol]
    qty = (accountval / len(tickers)) / price
    qty = int(qty)  # 取整数部分
    current_shares.append(qty)

# 打印当前股票数量
for i, symbol in enumerate(tickers):
    print(f"The current number of stock {symbol} : {current_shares[i]}")

# 计算最优股票数量与当前股票数量之间的差异
difference = [shares[i] - current_shares[i] for i in range(len(tickers))]

# 打印差异
for i, symbol in enumerate(tickers):
    print(f"The difference of stock {symbol} : {difference[i]}")

The current number of stock SLB : 2341
The current number of stock OXY : 2184
The current number of stock BP : 3679
The current number of stock WMB : 4277
The current number of stock BKR : 3838
The current number of stock APA : 3199
The current number of stock HES : 905
The difference of stock SLB : 2034
The difference of stock OXY : 626
The difference of stock BP : -2550
The difference of stock WMB : 1977
The difference of stock BKR : -1238
The difference of stock APA : -2898
The difference of stock HES : 275


In [48]:
# Execute Trades to accoujnt for the difference
for i, symbol in enumerate(tickers):
    qty_diff = difference[i]
    if qty_diff > 0:
        # 买入股票
        create_order(symbol, qty_diff, "buy", "market", "gtc")
    elif qty_diff < 0:
        # 卖出股票
        create_order(symbol, abs(qty_diff), "sell", "market", "gtc")